# Plant Disease Classification with ResNet50

This notebook demonstrates plant disease classification using transfer learning with ResNet50.

## Prerequisites
- Python 3.10.x
- Virtual environment (recommended)
- Internet connection for downloading dataset and models


In [1]:
"""
Comprehensive Dependency Checker
This cell checks all required dependencies before proceeding.
Run this cell first to ensure your environment is properly configured.
"""

import sys
import subprocess
import importlib

# Try to import packaging for version comparison
try:
    from packaging import version
    HAS_PACKAGING = True
except ImportError:
    HAS_PACKAGING = False
    print("⚠️  Note: 'packaging' module not available - version checking will be limited")

def check_python_version():
    """Check if Python version is compatible"""
    required_major, required_minor = 3, 10
    current_version = sys.version_info
    is_compatible = (current_version.major == required_major and 
                    current_version.minor >= required_minor)
    
    print(f"🐍 Python Version Check")
    print(f"   Current: {current_version.major}.{current_version.minor}.{current_version.micro}")
    print(f"   Required: {required_major}.{required_minor}.x")
    
    if is_compatible:
        print("   ✅ Python version is compatible!")
    else:
        print(f"   ❌ ERROR: Python {required_major}.{required_minor}.x is required!")
        raise RuntimeError(f"Python {required_major}.{required_minor}.x required, got {current_version.major}.{current_version.minor}")
    
    return is_compatible

def check_package(package_name, min_version=None, import_name=None):
    """Check if a package is installed and optionally verify version"""
    import_name = import_name or package_name
    try:
        module = importlib.import_module(import_name)
        installed_version = getattr(module, '__version__', 'unknown')
        
        if min_version and installed_version != 'unknown' and HAS_PACKAGING:
            try:
                if version.parse(installed_version) < version.parse(min_version):
                    print(f"   ⚠️  {package_name}: {installed_version} (requires >= {min_version})")
                    return False
            except:
                pass
        
        print(f"   ✅ {package_name}: {installed_version}")
        return True
    except ImportError:
        print(f"   ❌ {package_name}: NOT INSTALLED")
        return False

def install_package(package_spec):
    """Install a package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_spec], 
                            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except:
        return False

# Main dependency check
print("=" * 60)
print("🔍 COMPREHENSIVE DEPENDENCY CHECK")
print("=" * 60)

# Check Python version
check_python_version()
print()

# Check core dependencies
print("📦 Core Dependencies:")
dependencies = {
    'kagglehub': None,
    'numpy': '1.23.5',
    'tensorflow': '2.15.0',
    'PIL': None,  # Pillow
}

missing_packages = []
for pkg, min_ver in dependencies.items():
    if not check_package(pkg, min_ver):
        missing_packages.append(pkg)

print()

# Check optional dependencies
print("📦 Optional Dependencies:")
optional_deps = ['matplotlib', 'seaborn']
for pkg in optional_deps:
    check_package(pkg)

print()
print("=" * 60)

if missing_packages:
    print(f"⚠️  Missing packages: {', '.join(missing_packages)}")
    print("   Please install them using: %pip install <package_name>")
    print("   Or install all requirements: %pip install -r requirements.txt")
else:
    print("✅ All required dependencies are installed!")
    print("   You can proceed to the next cell.")

print("=" * 60)


🔍 COMPREHENSIVE DEPENDENCY CHECK
🐍 Python Version Check
   Current: 3.10.19
   Required: 3.10.x
   ✅ Python version is compatible!

📦 Core Dependencies:
   ✅ kagglehub: 0.3.13


/home/mahmoud-matter/hnu-projects/ai-skills/MachinLearningModels/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-12 15:46:24.906179: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


   ✅ numpy: 2.2.6


2025-12-12 15:46:24.925129: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-12 15:46:25.485048: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-12 15:46:28.834740: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-12 15:46:28.838298: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31]

   ✅ tensorflow: 2.20.0
   ✅ PIL: 12.0.0

📦 Optional Dependencies:
   ❌ matplotlib: NOT INSTALLED
   ❌ seaborn: NOT INSTALLED

✅ All required dependencies are installed!
   You can proceed to the next cell.


## Install and Import KaggleHub

Install kagglehub if not already installed, then download the plant disease dataset.

In [2]:
# Install kagglehub if not already installed
try:
    import kagglehub
    print("✅ kagglehub is already installed")
except ImportError:
    print("📦 Installing kagglehub...")
    %pip install -q kagglehub
    import kagglehub
    print("✅ kagglehub installed successfully")

✅ kagglehub is already installed


In [3]:
import kagglehub
import os

try:
    # Download latest version
    print("📥 Downloading plant disease dataset from Kaggle...")
    kaggle_path = kagglehub.dataset_download("emmarex/plantdisease")
    print(f"✅ Dataset downloaded to: {kaggle_path}")
    
    # Find the PlantVillage folder within the Kaggle dataset
    # The dataset might have the PlantVillage folder directly or inside a subfolder
    dataset_path = None
    
    # Check common locations
    possible_paths = [
        os.path.join(kaggle_path, "PlantVillage"),
        os.path.join(kaggle_path, "archive", "PlantVillage"),
        kaggle_path
    ]
    
    for path in possible_paths:
        if os.path.exists(path) and os.path.isdir(path):
            # Check if this looks like the PlantVillage folder (has subdirectories)
            subdirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
            if len(subdirs) > 5:  # PlantVillage should have many class folders
                dataset_path = path
                break
    
    # If not found, search recursively
    if dataset_path is None:
        print("🔍 Searching for PlantVillage folder...")
        for root, dirs, files in os.walk(kaggle_path):
            if "PlantVillage" in dirs:
                candidate = os.path.join(root, "PlantVillage")
                if os.path.isdir(candidate):
                    subdirs = [d for d in os.listdir(candidate) if os.path.isdir(os.path.join(candidate, d))]
                    if len(subdirs) > 5:
                        dataset_path = candidate
                        break
    
    if dataset_path is None:
        raise FileNotFoundError("Could not find PlantVillage folder in the downloaded dataset")
    
    # Validate dataset path
    if not os.path.exists(dataset_path):
        raise FileNotFoundError(f"Dataset path does not exist: {dataset_path}")
    
    if not os.path.isdir(dataset_path):
        raise NotADirectoryError(f"Dataset path is not a directory: {dataset_path}")
    
    print(f"✅ Found PlantVillage folder: {dataset_path}")
    
    # Count classes to verify dataset
    class_folders = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
    print(f"📊 Found {len(class_folders)} class folders")
    
except Exception as e:
    print(f"❌ Error downloading or locating dataset: {e}")
    raise

📥 Downloading plant disease dataset from Kaggle...
✅ Dataset downloaded to: /home/mahmoud-matter/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1
✅ Found PlantVillage folder: /home/mahmoud-matter/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1/PlantVillage
📊 Found 15 class folders


## Extract ZIP File (if needed)

Check if the dataset needs to be extracted from a ZIP file.

In [4]:
import zipfile
import os

try:
    # Check if we need to extract a zip file from the Kaggle dataset
    # Kaggle usually extracts automatically, but let's check for zip files
    if 'kaggle_path' not in globals():
        raise NameError("kaggle_path not defined. Please run the previous cell first.")
    
    if not os.path.exists(kaggle_path):
        raise FileNotFoundError(f"Kaggle path does not exist: {kaggle_path}")
    
    zip_files = [f for f in os.listdir(kaggle_path) if f.endswith('.zip')]
    
    if zip_files:
        print(f"📦 Found zip file(s): {zip_files}")
        zip_path = os.path.join(kaggle_path, zip_files[0])
        
        if not os.path.exists(zip_path):
            raise FileNotFoundError(f"Zip file not found: {zip_path}")
        
        extract_path = kaggle_path
        print(f"📂 Extracting {zip_files[0]}...")
        
        try:
            with zipfile.ZipFile(zip_path, 'r') as z:
                z.extractall(extract_path)
            print("✅ Zip file extracted successfully!")
        except zipfile.BadZipFile:
            raise ValueError(f"Invalid zip file: {zip_path}")
        except Exception as e:
            raise RuntimeError(f"Error extracting zip file: {e}")
        
        # Re-find PlantVillage after extraction
        print("🔍 Re-locating PlantVillage folder...")
        for root, dirs, files in os.walk(kaggle_path):
            if "PlantVillage" in dirs:
                candidate = os.path.join(root, "PlantVillage")
                if os.path.isdir(candidate):
                    dataset_path = candidate
                    break
        
        if 'dataset_path' in locals() and dataset_path:
            print(f"✅ Updated dataset path: {dataset_path}")
        else:
            raise FileNotFoundError("Could not find PlantVillage folder after extraction")
    else:
        print("✅ No zip files found - dataset is already extracted from Kaggle")
        if 'dataset_path' in globals():
            print(f"✅ Using dataset path: {dataset_path}")
        else:
            raise NameError("dataset_path not defined. Please run the previous cell first.")
            
except Exception as e:
    print(f"❌ Error in zip extraction: {e}")
    raise

✅ No zip files found - dataset is already extracted from Kaggle
✅ Using dataset path: /home/mahmoud-matter/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1/PlantVillage


List Files in a Folder 📂

Quickly see all files in a directory

In [5]:
import os

# Use the Kaggle dataset path (defined in cell 2)
print(os.listdir(dataset_path))


['Potato___Early_blight', 'Tomato__Tomato_mosaic_virus', 'Tomato_Late_blight', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Target_Spot', 'Potato___healthy', 'Tomato_Septoria_leaf_spot', 'Tomato_Early_blight', 'Potato___Late_blight', 'Tomato_healthy', 'Tomato_Bacterial_spot', 'Tomato_Leaf_Mold', 'Pepper__bell___healthy', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Pepper__bell___Bacterial_spot']


 Explore Folder Structure in Python 📂

Print folder paths and a few files inside each folder:

In [6]:
import os

# Use the Kaggle dataset path (defined in cell 2)
root = dataset_path

for path, dirs, files in os.walk(root):
    print(f"📁 Folder: {path}")
    for f in files[:5]: 
        print("   -", f)
    print("---------------")


📁 Folder: /home/mahmoud-matter/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1/PlantVillage
---------------
📁 Folder: /home/mahmoud-matter/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1/PlantVillage/Potato___Early_blight
   - 583fa343-d5be-414b-bd95-f674313bd882___RS_Early.B 7681.JPG
   - 06ac6596-8d65-46dd-a343-a2209f3480e4___RS_Early.B 6921.JPG
   - b35e4c48-e54e-4e42-a399-b236209314f4___RS_Early.B 8231.JPG
   - b05850b8-bfb9-487d-b0ba-8f406eba933e___RS_Early.B 6936.JPG
   - c10de41b-4c38-41c4-b278-600071051453___RS_Early.B 8093.JPG
---------------
📁 Folder: /home/mahmoud-matter/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1/PlantVillage/Tomato__Tomato_mosaic_virus
   - 7a6e4837-d63d-465a-abae-ef857bf43e2d___PSU_CG 2117.JPG
   - b5987570-3fbe-4a4c-9790-1d55bcfd2c61___PSU_CG 2288.JPG
   - 86965ac5-b790-4218-a3c6-216d10da5211___PSU_CG 2066.JPG
   - 9ec78399-de3a-4431-a5fc-e100406c5642___PSU_CG 2275.JPG
   - 9a3941e6-32bc-4417-862b-2f03e4040b65___PSU_CG 2

Count Images in Each Class Folder 🖼️

Check how many images are in each subfolder

In [7]:
import os

# Use the Kaggle dataset path (defined in cell 2)
base_path = dataset_path

for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path):
        count = len(os.listdir(folder_path))
        print(f"{folder}: {count} images")


Potato___Early_blight: 1000 images
Tomato__Tomato_mosaic_virus: 373 images
Tomato_Late_blight: 1909 images
Tomato__Tomato_YellowLeaf__Curl_Virus: 3209 images
Tomato__Target_Spot: 1404 images
Potato___healthy: 152 images
Tomato_Septoria_leaf_spot: 1771 images
Tomato_Early_blight: 1000 images
Potato___Late_blight: 1000 images
Tomato_healthy: 1591 images
Tomato_Bacterial_spot: 2127 images
Tomato_Leaf_Mold: 952 images
Pepper__bell___healthy: 1478 images
Tomato_Spider_mites_Two_spotted_spider_mite: 1676 images
Pepper__bell___Bacterial_spot: 997 images


In [8]:
# Enhanced Python and TensorFlow Compatibility Check
import sys

def check_tensorflow_compatibility():
    """Check Python version and TensorFlow installation"""
    required_major, required_minor = 3, 10
    current_version = sys.version_info
    
    print("=" * 60)
    print("🔍 TensorFlow Compatibility Check")
    print("=" * 60)
    
    # Check Python version
    print(f"🐍 Python Version: {current_version.major}.{current_version.minor}.{current_version.micro}")
    
    if current_version.major == required_major and current_version.minor >= required_minor:
        print("   ✅ Python version is compatible with TensorFlow")
    else:
        print(f"   ❌ ERROR: Python {required_major}.{required_minor}.x is required!")
        raise RuntimeError(f"Python {required_major}.{required_minor}.x required for TensorFlow")
    
    # Check TensorFlow installation
    print("\n📦 TensorFlow Check:")
    try:
        import tensorflow as tf
        tf_version = tf.__version__
        print(f"   ✅ TensorFlow {tf_version} is installed")
        
        # Check TensorFlow version compatibility
        try:
            from packaging import version
            if version.parse(tf_version) >= version.parse("2.15.0"):
                print("   ✅ TensorFlow version is compatible")
            else:
                print(f"   ⚠️  WARNING: TensorFlow 2.15.0+ recommended, found {tf_version}")
        except:
            pass
        
        # Check GPU availability
        print("\n🖥️  Hardware Check:")
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            print(f"   ✅ Found {len(gpus)} GPU(s)")
            for i, gpu in enumerate(gpus):
                print(f"      GPU {i}: {gpu.name}")
        else:
            print("   ℹ️  No GPU found - will use CPU (training will be slower)")
        
        return True
    except ImportError:
        print("   ❌ TensorFlow is NOT installed")
        print("   📦 Please install TensorFlow: %pip install tensorflow>=2.15.0")
        return False

# Run compatibility check
tensorflow_ready = check_tensorflow_compatibility()
print("=" * 60)

if not tensorflow_ready:
    raise ImportError("TensorFlow is not installed. Please install it before proceeding.")


🔍 TensorFlow Compatibility Check
🐍 Python Version: 3.10.19
   ✅ Python version is compatible with TensorFlow

📦 TensorFlow Check:
   ✅ TensorFlow 2.20.0 is installed
   ✅ TensorFlow version is compatible

🖥️  Hardware Check:
   ℹ️  No GPU found - will use CPU (training will be slower)


2025-12-12 15:46:30.731775: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


## Install TensorFlow (if needed)

Install TensorFlow if it's not already installed. This may take several minutes.

In [9]:
# Install TensorFlow 2.15+ - Requires Python 3.10.x
# Make sure you've run the Python version check cell above first!

try:
    import tensorflow as tf
    print(f"✅ TensorFlow {tf.__version__} is already installed")
    print("   Skipping installation.")
except ImportError:
    print("📦 TensorFlow not found. Installing TensorFlow 2.15.0...")
    print("   This may take several minutes. Please be patient...")
    try:
        %pip install -q tensorflow>=2.15.0
        import tensorflow as tf
        print(f"✅ TensorFlow {tf.__version__} installed successfully!")
    except Exception as e:
        print(f"❌ Error installing TensorFlow: {e}")
        print("   Please try installing manually: %pip install tensorflow>=2.15.0")
        raise


✅ TensorFlow 2.20.0 is already installed
   Skipping installation.


In [10]:
import sys
print(sys.version)


3.10.19 (main, Oct 10 2025, 08:52:10) [GCC 13.3.0]


In [11]:
!python -m pip install --upgrade pip


In [12]:
!python -m pip install --upgrade pip setuptools wheel



In [13]:
!python -m pip install tensorflow


Import TensorFlow and OS Modules ⚡

In [14]:
import tensorflow as tf
import os

Load Plant Images as TensorFlow Datasets 🌱🖼️

In [15]:
import tensorflow as tf
import os

# Validate dataset_path exists
if 'dataset_path' not in globals():
    raise NameError("dataset_path not defined. Please run the dataset download cell first.")

if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset path does not exist: {dataset_path}")

# Configuration
img_size = (224, 224)
batch_size = 32
validation_split = 0.2
random_seed = 42

print("📂 Loading plant images as TensorFlow datasets...")
print(f"   Dataset path: {dataset_path}")
print(f"   Image size: {img_size}")
print(f"   Batch size: {batch_size}")
print(f"   Validation split: {validation_split * 100}%")

try:
    # Load training dataset
    print("\n📥 Loading training dataset...")
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        dataset_path,
        validation_split=validation_split,
        subset="training",
        seed=random_seed,
        image_size=img_size,
        batch_size=batch_size,
        label_mode='int'  # Explicit label mode
    )
    
    # IMPORTANT: Save class_names BEFORE any transformations
    class_names = train_ds.class_names
    num_classes = len(class_names)
    
    print(f"✅ Training dataset loaded:")
    print(f"   - Found {len(train_ds.file_paths) if hasattr(train_ds, 'file_paths') else 'N/A'} files")
    print(f"   - Number of classes: {num_classes}")
    print(f"   - Classes: {class_names}")
    
    # Load validation dataset
    print("\n📥 Loading validation dataset...")
    val_test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        dataset_path,
        validation_split=validation_split,
        subset="validation",
        seed=random_seed,
        image_size=img_size,
        batch_size=batch_size,
        label_mode='int'  # Explicit label mode
    )
    
    print(f"✅ Validation dataset loaded:")
    print(f"   - Found {len(val_test_ds.file_paths) if hasattr(val_test_ds, 'file_paths') else 'N/A'} files")
    
    # Verify class names match
    if val_test_ds.class_names != class_names:
        print("⚠️  WARNING: Class names don't match between train and validation sets!")
    else:
        print("✅ Class names verified (train and validation match)")
    
except Exception as e:
    print(f"❌ Error loading datasets: {e}")
    raise


📂 Loading plant images as TensorFlow datasets...
   Dataset path: /home/mahmoud-matter/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1/PlantVillage
   Image size: (224, 224)
   Batch size: 32
   Validation split: 20.0%

📥 Loading training dataset...
Found 20638 files belonging to 15 classes.
Using 16511 files for training.
✅ Training dataset loaded:
   - Found 16511 files
   - Number of classes: 15
   - Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']

📥 Loading validation dataset...
Found 20638 files belonging to 15 classes.
Using 4127 files for validation.
✅ Validation dataset loaded:
   - Found 4127 files
✅ Clas

In [16]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])


In [17]:
# Optimize dataset performance with prefetching
# NOTE: class_names and num_classes should already be saved from the previous cell

AUTOTUNE = tf.data.AUTOTUNE

# Verify class_names and num_classes are defined
if 'class_names' not in globals() or 'num_classes' not in globals():
    raise NameError("class_names and num_classes not defined. Please run the dataset loading cell first.")

print(f"📊 Dataset Summary:")
print(f"   - Number of classes: {num_classes}")
print(f"   - Class names: {class_names}")

# Apply prefetching for better performance
# This must be done AFTER saving class_names (which we did in the previous cell)
print("\n⚡ Optimizing dataset performance with prefetching...")
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_test_ds = val_test_ds.prefetch(buffer_size=AUTOTUNE)

print("✅ Datasets optimized and ready for training")


📊 Dataset Summary:
   - Number of classes: 15
   - Class names: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']

⚡ Optimizing dataset performance with prefetching...
✅ Datasets optimized and ready for training


In [18]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, Model

# Verify required variables are defined
required_vars = ['num_classes', 'class_names', 'data_augmentation']
missing_vars = [v for v in required_vars if v not in globals()]

if missing_vars:
    raise NameError(f"Missing required variables: {missing_vars}. Please run previous cells first.")

print("🏗️  Building ResNet50 Transfer Learning Model...")
print(f"   - Number of classes: {num_classes}")
print(f"   - Input shape: (224, 224, 3)")

try:
    # Load pre-trained ResNet50 base model
    print("\n📥 Loading pre-trained ResNet50 (ImageNet weights)...")
    base_model = ResNet50(
        weights="imagenet",
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Freeze base model layers
    base_model.trainable = False
    print("✅ Base model loaded and frozen")
    
    # Build the model
    print("\n🔨 Building model architecture...")
    inputs = layers.Input(shape=(224, 224, 3))
    
    # Apply data augmentation first (works on [0, 1] range)
    x = data_augmentation(inputs)
    
    # Scale from [0, 1] to [0, 255] for ResNet preprocessing
    x = layers.Rescaling(255.0)(x)
    
    # Apply ResNet preprocessing (normalizes to ImageNet stats)
    x = tf.keras.applications.resnet.preprocess_input(x)
    
    # Pass through base model
    x = base_model(x, training=False)
    
    # Add classification head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    
    model_resnet = Model(inputs, outputs)
    
    # Compile the model
    print("\n⚙️  Compiling model...")
    model_resnet.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    print("✅ Model compiled successfully!")
    print("\n📋 Model Summary:")
    model_resnet.summary()
    
except Exception as e:
    print(f"❌ Error building model: {e}")
    raise


🏗️  Building ResNet50 Transfer Learning Model...
   - Number of classes: 15
   - Input shape: (224, 224, 3)

📥 Loading pre-trained ResNet50 (ImageNet weights)...


✅ Base model loaded and frozen

🔨 Building model architecture...

⚙️  Compiling model...
✅ Model compiled successfully!

📋 Model Summary:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ sequential[0][0]  │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224)  │          0 │ rescaling[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224)  │          0 │ rescaling[0][0]   │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 224, 224)  │          0 │ rescaling[0][0]   │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 224, 224,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 224, 224,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ add[0][0]         │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 2048)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 15)        │     30,735 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,618,447 (90.10 MB)

 Trainable params: 30,735 (120.06 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
history_resnet = model_resnet.fit(
    train_ds,
    validation_data=val_test_ds,
    epochs=10
)


Epoch 1/10
120/516 ━━━━━━━━━━━━━━━━━━━━ 10:40 2s/step - accuracy: 0.1054 - loss: 635.9990

In [ ]:
import os

# Create models directory if it doesn't exist
models_dir = "models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"📁 Created directory: {models_dir}")

# Save the model
model_path = os.path.join(models_dir, "resnet50.h5")

try:
    if 'model_resnet' not in globals():
        raise NameError("model_resnet not defined. Please train the model first.")
    
    print(f"💾 Saving model to: {model_path}")
    model_resnet.save(model_path)
    print("✅ Model saved successfully!")
    
    # Verify the file was created
    if os.path.exists(model_path):
        file_size = os.path.getsize(model_path) / (1024 * 1024)  # Size in MB
        print(f"   File size: {file_size:.2f} MB")
    else:
        print("⚠️  WARNING: Model file not found after saving!")
        
except Exception as e:
    print(f"❌ Error saving model: {e}")
    raise


In [ ]:
print("changes")